# Data Generation

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import mysql.connector

# Create connection
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="1234567890",
    port=3306,
    database="pilot"
)

cursor = conn.cursor()

### Table Defined: users

In [2]:
# Create a DataFrame with user data
data = [
    {
        "firstname": "Alice",
        "lastname": "Smith",
        "phone_number": "123-456-7890",
        "email": "alice.smith@example.com",
        "username": "alice",
        "password": "0000",
        "card_id": "CARD001",
        "cash": 50000,
        "created_date": datetime(2024, 6, 1)
    },
    {
        "firstname": "Bob",
        "lastname": "Johnson",
        "phone_number": "234-567-8901",
        "email": "bob.johnson@example.com",
        "username": "bob",
        "password": "1111",
        "card_id": "CARD002",
        "cash": 15000,
        "created_date": datetime(2024, 6, 2)
    },
    {
        "firstname": "Charlie",
        "lastname": "Williams",
        "phone_number": "345-678-9012",
        "email": "charlie.williams@example.com",
        "username": "charlie",
        "password": "2222",
        "card_id": "CARD003",
        "cash": 30000,
        "created_date": datetime(2024, 6, 3)
    }
]
df_users = pd.DataFrame(data)

### Table Created: users

In [3]:

# Create users table if not exists
cursor.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INT AUTO_INCREMENT PRIMARY KEY,
        firstname VARCHAR(255),
        lastname VARCHAR(255),
        phone_number VARCHAR(255),
        email VARCHAR(255),
        username VARCHAR(255),
        password VARCHAR(255),
        card_id VARCHAR(255),
        cash FLOAT,
        created_date DATETIME
    )
""")

# Insert data
insert_query = """
    INSERT INTO users (firstname, lastname, phone_number, email, username, password, card_id, cash, created_date)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Convert DataFrame to list of tuples for insertion
values = df_users.values.tolist()

# --- User insert code (do NOT close connection here) ---
cursor.executemany(insert_query, values)
conn.commit()
# Do NOT close here


### Table Defined: stocks

In [4]:
from pathlib import Path

# List all files in rawdata folder
file_list = list(Path('rawdata').glob('*_data*'))

# Create an empty list to store dataframes
dfs = []

# Read each file and add filename as a column
for file in file_list:
    # Extract the filename part before '_data'
    source = file.stem.split('_data')[0]
    
    # Read the file
    temp_df = pd.read_csv(file)
    
    # Add source column
    temp_df['source'] = source

    # Create a mapping dictionary for source names
    source_name_mapping = {
        'tencent': 'Tencent Holdings',
        'nasdaq': 'NASDAQ Composite',
        'russell2000': 'Russell 2000',
        'samsung': 'Samsung Electronics',
        'google': 'Alphabet Inc.',
        'dowjones': 'Dow Jones',
        'tsmc': 'TSMC',
        'baidu': 'Baidu Inc.',
        'sp500': 'S&P 500',
        'amazon': 'Amazon',
        'nvidia': 'NVIDIA Corp.',
        'hangseng': 'Hang Seng Index',
        'csi300': 'CSI 300',
        'apple': 'Apple Inc.',
        'facebook': 'Meta Platforms',
        'alibaba': 'Alibaba Group',
        'microsoft': 'Microsoft Corp.',
        'tesla': 'Tesla Inc.'
    }

    # Add the source_name column
    temp_df['source_name'] = temp_df['source'].map(source_name_mapping)
    
    # Append to list
    dfs.append(temp_df)

# Combine all dataframes
if dfs:
    combined_df = pd.concat(dfs, ignore_index=True)
else:
    print("No files found in rawdata folder")

# Remove records with negative numbers in any of the numeric columns
numeric_cols = combined_df.select_dtypes(include=['float64', 'int64']).columns
combined_df = combined_df[~(combined_df[numeric_cols] < 0).any(axis=1)]

### Table Created: stocks

In [5]:
# Create stocks table if not exists
cursor.execute("""
    CREATE TABLE IF NOT EXISTS stocks (
        id INT AUTO_INCREMENT PRIMARY KEY,
        date DATETIME,
        open FLOAT,
        high FLOAT,
        low FLOAT,
        close FLOAT,
        volume BIGINT,
        dividends FLOAT,
        stock_splits FLOAT,
        source VARCHAR(255),
        source_name VARCHAR(255)
    )
""")

# Insert data
insert_query = """
    INSERT INTO stocks (date, open, high, low, close, volume, dividends, stock_splits, source, source_name)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Convert DataFrame to list of tuples for insertion
values = combined_df.values.tolist()

cursor.executemany(insert_query, values)
conn.commit()

### Bridge Tables: collections

In [6]:
import random

# Create collections table that links users and stocks
cursor.execute("""
    CREATE TABLE IF NOT EXISTS collections (
        id INT AUTO_INCREMENT PRIMARY KEY,
        user_id INT,
        stock_name VARCHAR(255),
        created_date DATETIME DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (user_id) REFERENCES users(id),
        UNIQUE KEY unique_collection (user_id, stock_name)
    )
""")

conn.commit()

# Get all user IDs
cursor.execute("SELECT id FROM users")
user_ids = [row[0] for row in cursor.fetchall()]

# Get unique stock names from combined_df
stock_names = combined_df['source_name'].unique().tolist()

# Generate collections for each user
collection_data = []
for user_id in user_ids:
    # Randomly choose 2-4 unique stocks for each user
    num_stocks = random.randint(2, 4)
    user_stocks = random.sample(stock_names, num_stocks)
    
    for stock_name in user_stocks:
        collection_data.append((user_id, stock_name))

# Insert the records
insert_query = """
    INSERT INTO collections (user_id, stock_name)
    VALUES (%s, %s)
"""
cursor.executemany(insert_query, collection_data)
conn.commit()


### Bridge Table: trades

In [ ]:
# Create trades table if not exists
cursor.execute("""
    CREATE TABLE IF NOT EXISTS trades (
        id INT AUTO_INCREMENT PRIMARY KEY,
        user_id INT,
        stock_id INT,
        source_name VARCHAR(255),
        action VARCHAR(10),
        price FLOAT,
        quantity INT,
        total_amount FLOAT,
        created_date DATETIME,
        FOREIGN KEY (user_id) REFERENCES users(id),
        FOREIGN KEY (stock_id) REFERENCES stocks(id)
    )
""")

# Get stock prices for trades (include date, close price, and source_name)
cursor.execute("SELECT id, source_name, date, close FROM stocks ORDER BY date")
stock_records = cursor.fetchall()

trade_data = []
for _ in range(5):
    user_id = random.choice(user_ids)
    # Pick a random stock record that includes id, source_name, date and price
    stock_record = random.choice(stock_records)
    stock_id = stock_record[0]
    source_name = stock_record[1]
    date = stock_record[2]
    price = stock_record[3]
    
    action = random.choice(['buy', 'sell'])
    quantity = random.randint(1, 100)
    total_amount = price * quantity
    
    trade_data.append((user_id, stock_id, source_name, action, price, quantity, total_amount, date))

# Insert trades
insert_query = """
    INSERT INTO trades (user_id, stock_id, source_name, action, price, quantity, total_amount, created_date)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""
cursor.executemany(insert_query, trade_data)
conn.commit()